# Strands Agents 세션 간 메모리 지속하기

이 예제에서는 Strands Agents에서 다양한 세션 간에 메모리를 지속하는 방법을 배우게 됩니다.

`duckduckgo` 검색 API를 사용하여 웹 검색을 수행하는 에이전트의 사용 사례를 사용하겠습니다.

이 노트북에서는 다음을 수행합니다:
- 메모리 기반 Strands 에이전트의 기능을 탐색합니다.
- 메모리를 저장, 검색 및 나열하는 방법을 배웁니다.
- 에이전트를 통해 웹 검색을 수행하는 방법을 이해합니다.
- 대화형 루프에서 에이전트와 상호작용합니다.


### 사용 예시

메모리 저장:
```
내가 커피보다 차를 선호한다는 것을 기억해주세요
```

메모리 검색:
```
내가 마시기를 선호하는 것은 무엇인가요?
```

모든 메모리 나열:
```
나에 대해 기억하는 모든 것을 보여주세요
```

### 메모리 사용 팁

- 에이전트에게 정보를 기억하도록 요청할 때 명시적으로 표현하세요
- 관련 메모리를 검색하기 위해 구체적인 쿼리를 사용하세요
- 메모리 지속성은 더 자연스럽고 맥락적인 대화를 가능하게 합니다

## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정 및 환경에 구성된 AWS 자격 증명
* Amazon Bedrock에서 Anthropic Claude 3.7 활성화
* Amazon Bedrock Knowledge Base, Amazon S3 버킷 및 Amazon DynamoDB 생성 권한이 있는 IAM 역할

이제 Strands Agent에 필요한 패키지들을 설치하겠습니다

In [ ]:
# 필요한 패키지 설치
%pip install -r requirements.txt

In [ ]:
# 필요한 라이브러리 가져오기
import logging
from strands import Agent, tool
from strands_tools import memory
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

## 도구 설정

웹 검색을 위한 사용자 정의 도구를 정의하겠습니다.

In [ ]:
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """웹을 검색하여 업데이트된 정보를 가져옵니다.
    
    Args:
        keywords (str): 검색 쿼리 키워드
        region (str): 검색 지역 (기본값: us-en)
        max_results (int): 반환할 최대 결과 수 (기본값: 5)
    
    Returns:
        str: 검색 결과
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        if results:
            formatted_results = []
            for i, result in enumerate(results, 1):
                formatted_results.append(
                    f"{i}. {result.get('title', 'N/A')}\n"
                    f"   URL: {result.get('href', 'N/A')}\n"
                    f"   요약: {result.get('body', 'N/A')[:200]}...\n"
                )
            return "\n".join(formatted_results)
        else:
            return "검색 결과를 찾을 수 없습니다."
    except RatelimitException:
        return "속도 제한 예외: 잠시 후 다시 시도해주세요."
    except DDGSException as d:
        return f"DuckDuckGo 검색 예외: {d}"
    except Exception as e:
        return f"예외 발생: {e}"

## 메모리 기능이 있는 에이전트 생성

이제 메모리와 웹 검색 기능을 모두 갖춘 에이전트를 생성하겠습니다.

In [ ]:
# 로깅 구성
logging.getLogger("strands").setLevel(logging.INFO)

# 시스템 프롬프트 정의
system_prompt = """
당신은 메모리 기능과 웹 검색 기능을 갖춘 도움이 되는 개인 어시스턴트입니다.

주요 기능:
1. 사용자에 대한 정보를 기억하고 저장할 수 있습니다
2. 이전에 저장된 메모리를 검색할 수 있습니다
3. 웹 검색을 통해 최신 정보를 찾을 수 있습니다

메모리 사용 가이드라인:
- 사용자가 "기억해주세요" 또는 "저장해주세요"라고 말하면 해당 정보를 메모리에 저장하세요
- 사용자에 대한 개인적인 정보나 선호도를 자동으로 기억하세요
- 관련 메모리가 있을 때 자연스럽게 활용하세요

항상 도움이 되고 친근한 방식으로 응답해주세요.
"""

# 메모리와 웹 검색 도구를 포함한 에이전트 생성
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=system_prompt,
    tools=[memory, websearch],
)

## 에이전트 테스트

메모리 기능과 웹 검색 기능을 테스트해보겠습니다.

In [ ]:
# 테스트 1: 정보 저장
response = agent("내 이름은 김철수이고, 서울에 살고 있습니다. 그리고 파이썬 프로그래밍을 좋아합니다. 이것을 기억해주세요.")
print(response)

In [ ]:
# 테스트 2: 저장된 정보 검색
response = agent("내 이름이 뭐였죠?")
print(response)

In [ ]:
# 테스트 3: 웹 검색
response = agent("파이썬의 최신 버전에 대해 검색해주세요.")
print(response)

In [ ]:
# 테스트 4: 메모리와 검색 결합
response = agent("제가 좋아하는 프로그래밍 언어의 최신 뉴스를 찾아주세요.")
print(response)

## 대화형 루프

에이전트와 대화형으로 상호작용할 수 있는 루프를 만들어보겠습니다.

In [ ]:
def interactive_chat():
    """에이전트와의 대화형 채팅 세션을 시작합니다."""
    print("메모리 기능이 있는 개인 어시스턴트와 대화를 시작합니다.")
    print("종료하려면 'quit', 'exit', 또는 'bye'를 입력하세요.\n")
    
    while True:
        try:
            user_input = input("사용자: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye', '종료']:
                print("어시스턴트: 안녕히 가세요! 대화 내용은 메모리에 저장되었습니다.")
                break
            
            if not user_input:
                continue
            
            response = agent(user_input)
            print(f"어시스턴트: {response}\n")
            
        except KeyboardInterrupt:
            print("\n어시스턴트: 대화가 중단되었습니다. 안녕히 가세요!")
            break
        except Exception as e:
            print(f"오류가 발생했습니다: {e}")

# 대화형 채팅 시작 (주석을 해제하여 실행)
# interactive_chat()

## 결론

이 노트북에서는 Strands Agents의 메모리 지속 기능을 배웠습니다. 주요 학습 내용:

1. **메모리 도구 사용**: `memory` 도구를 통한 정보 저장 및 검색
2. **웹 검색 통합**: DuckDuckGo API를 사용한 실시간 정보 검색
3. **세션 간 지속성**: 에이전트가 이전 대화 내용을 기억하고 활용
4. **자연스러운 상호작용**: 메모리를 활용한 맥락적 대화

### 활용 방안

- **개인 어시스턴트**: 사용자 선호도와 정보를 기억하는 맞춤형 어시스턴트
- **고객 서비스**: 고객 이력과 선호도를 기억하는 서비스 봇
- **학습 도우미**: 학습 진도와 약점을 기억하는 교육용 에이전트

메모리 기능을 통해 더욱 개인화되고 효과적인 AI 에이전트를 구축할 수 있습니다!